In [8]:
from shapely.geometry import Point
import geopandas as gpd
from pykml.factory import KML_ElementMaker as KML
from lxml import etree
import pandas as pd
from shapely.geometry import Polygon

In [9]:
lat_start, lat_end = 40.80, 41.62
lon_start, lon_end = 27.94, 29.97
lat_step = 0.02
lon_step = 0.03

lat_points = [round(lat_start + i * lat_step, 4) for i in range(int((lat_end - lat_start) / lat_step) + 1)]
lon_points = [round(lon_start + i * lon_step, 4) for i in range(int((lon_end - lon_start) / lon_step) + 1)]

grid_points = [(lat, lon) for lat in lat_points for lon in lon_points]

In [10]:
def create_kml(gdf, filename):
    placemarks = []
    for _, row in gdf.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        placemarks.append(
            KML.Placemark(
                KML.name(f"{lat}, {lon}"),
                KML.Point(KML.coordinates(f"{lon},{lat},0"))
            )
        )

    kml_doc = KML.kml(KML.Document(*placemarks))
    with open(filename, "w", encoding="utf-8") as f:
        f.write(etree.tostring(kml_doc, pretty_print=True).decode("utf-8"))


In [11]:
geoms = [Point(lon, lat) for lat, lon in grid_points]
gdf = gpd.GeoDataFrame(geometry=geoms, crs="EPSG:4326")

create_kml(gdf, "assets/istanbul_grid.kml")

In [12]:
polygon_df = pd.read_csv("assets/Istanbul.csv")
polygon_list = []
for row in polygon_df.iloc[:, 0]:
    coords = [(float(p.split(",")[0]), float(p.split(",")[1])) for p in row.strip().split()]
    polygon_list.append(Polygon(coords))

gdf["is_inside"] = gdf.geometry.apply(lambda point: any(point.within(poly) for poly in polygon_list))

inside_points = gdf[gdf["is_inside"]].copy()

# KML
create_kml(inside_points, "assets/istanbul_filtered_grid.kml")

# CSV
inside_points["lon"] = inside_points.geometry.x
inside_points["lat"] = inside_points.geometry.y
inside_points[["lat", "lon"]].to_csv("assets/istanbul_filtered_grid.csv", index=False)


In [13]:
print(len(inside_points))
print(len(grid_points))

973
2856
